In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
print("Dataset shape:", dataset.shape)
print("\nColumn names and types:")
print(dataset.dtypes)
print("\nFirst few rows:")
print(dataset.head())
print("\nChecking for missing values to confirm if thier is :")
print(dataset.isnull().sum())
print("\n the old shape:", dataset.shape)




In [ ]:
dataset['sex'] = dataset['sex'].map({'male': 1, 'female': 0})
dataset['smoker'] = dataset['smoker'].map({'yes': 1, 'no': 0})

dataset = pd.get_dummies(dataset, columns=['region'], prefix='region')

print("\n Dataset after encoding:")
print(dataset.head())
print("\n the new shape:", dataset.shape)


In [ ]:
# prepering the train / test dataset i will give 80% train / 20 test


train_dataset = dataset.sample(frac=0.8, random_state=42)
test_dataset = dataset.drop(train_dataset.index)



print(f"\n training set size: {len(train_dataset)}")
print(f"Test set size: {len(test_dataset)}")


In [ ]:
# specifie labels

train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')

print(f"\nTraining features shape: {train_dataset.shape}")
print(f"Training labels shape: {train_labels.shape}")
print(f"Test features shape: {test_dataset.shape}")
print(f"Test labels shape: {test_labels.shape}")

In [ ]:
# Normalize the data

normalizer = layers.Normalization(axis=-1)
normalizer.adapt(np.array(train_dataset))

# take a look at the stats
print(f"Mean: {normalizer.mean.numpy()}")
print(f"Variance: {normalizer.variance.numpy()}")

In [ ]:
# Build the NN model

model = keras.Sequential([
    normalizer,
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

# Using Mean Absolute Error (MAE) as the loss function for regression

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mean_absolute_error',
    metrics=['mean_absolute_error', 'mean_squared_error']
)


print("\n Model summary:")
model.summary()


In [ ]:
#i will use this to prevent overfitting to restore the best weight
early_stop = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=50,
    restore_best_weights=True
)


history = model.fit(
    train_dataset,
    train_labels,
    epochs=1000,
    validation_split=0.2,
    verbose=0,
    callbacks=[early_stop, tfdocs.modeling.EpochDots()]
)

print("\n \n \n Training complete!")

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
